In [ ]:
%pip install -e ..

In [ ]:
%pip install gym
%pip install optuna
%pip install rl_zoo3

In [1]:
import numpy as np
import pandas as pd
from plotnine import ggplot, aes, geom_density, geom_line, geom_point, geom_violin, facet_grid, labs, theme, facet_wrap

from stable_baselines3 import PPO, TD3
from sb3_contrib import TQC
from stable_baselines3.common.env_util import make_vec_env

from rl4greencrab import greenCrabSimplifiedEnv, simulator
import gym

import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances

In [2]:
gcse = greenCrabSimplifiedEnv()
vec_env = make_vec_env(greenCrabSimplifiedEnv, n_envs=12)
eval_envs = vec_env

In [3]:
model = PPO("MlpPolicy", vec_env, verbose=0, 
            batch_size =  64, # no
            n_steps = 32, # no
            gamma = 0.95, # no
            learning_rate =  0.1, # orginal = 0.9447561168646137
            ent_coef = 7.676877009214456e-07, # no
            clip_range =  0.1, # no
            n_epochs = 20, 
            gae_lambda = 0.9, # no
            max_grad_norm = 5, # no
            vf_coef = 0.9694540149524578, # no
            tensorboard_log="/home/rstudio/logs")
model.learn(
	total_timesteps=250_000, 
	progress_bar=True,
)
model.save("ppo_gcse")

Output()

In [5]:
model = PPO("MlpPolicy", vec_env, verbose=0, 
            learning_rate =  0.1, # orginal = 0.9447561168646137
            tensorboard_log="/home/rstudio/logs")
model.learn(
	total_timesteps=250_000, 
	progress_bar=True,
)
model.save("ppo_gcse")

Output()